# Neural Networks Official Tutorial

Neural networks can be constructed using the `torch.nn` package.

`nn` depends on `autograd` to define models and differentiate them. An `nn.Module` contains layers, and a method `forward(input)` that returns the `output`.

For example, look at this network:

<img src="../Image/mnist.png" width="100%">

A typical training procedure for a neural network is as follow:

- Define the neural network that has some learnable parameters (or weights)

- Iterate over a dataset of inputs

- Process input through the network

- Compute the loss (how far is the output from being correct)

- Propagate gradients back into the network's parameters

- Update the weights of the network, typically using a simple update rule: `weight = weight - learning_rate * gradient`

## Define the network

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3*3 convolutional kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        # the second convolutional layer
        self.conv2 = nn.Conv2d(6, 16, 3)
        # An affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)     # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]     # all dimensions except the batch dimension
        print("size = ", size)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the `forward` function, and the `backward` function (where gradients are computed) is automatically defined for you using `autograd`. You can use any of the Tensor operations in the `forward` function.

**params_number = (shape of width of the filter * shape of height of the filter + 1) * number of filters**

In [9]:
params = list(net.parameters())
print(len(params))
print(params[0].size())     # conv1's .weight

10
torch.Size([6, 1, 3, 3])


Let's try a random 32x32 input. Note: expected input size of this net (LeNet) is 32x32. To use this net on MNIST dataset, please resize the images from the dataset to 32x32.

In [11]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

size =  torch.Size([16, 6, 6])
tensor([[-0.0954, -0.0791,  0.0810, -0.0683,  0.0174,  0.0360,  0.0779, -0.0258,
          0.0806, -0.0593]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random gradients:

In [13]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## Loss Function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different **loss functions** under the nn package . A simple loss is: `nn.MSELoss` which computes the mean-squared error between the input and the target.

In [15]:
output = net(input)
target = torch.randn(10)     # a dummy target, for example
target = target.view(1, -1)     # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

size =  torch.Size([16, 6, 6])
tensor(0.8859, grad_fn=<MseLossBackward>)


In [18]:
"""
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
"""

'\ninput -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d\n      -> view -> linear -> relu -> linear -> relu -> linear\n      -> MSELoss\n      -> loss\n'

In [19]:
print(loss.grad_fn)     # MSELoss
print(loss.grad_fn.next_functions[0][0])     # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])     # ReLU

## Backprop

To backpropagate the error all we have to do is to `loss.backward()`. You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

Now we shall call `loss.backward()`, and have a look at conv1’s bias gradients before and after the backward.

In [31]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

# loss.backward(retain_graph=True)

# print('conv1.bias.grad after backward')
# print(net.conv1.bias.grad)

conv1.bias.grad before backward
None


## Update the weights

The simple update rule used in practice is the Stochastic Gradient Descent (SGD):
`weight = weight - learning_rate * gradient`

In [36]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: `torch.optim` that implements all these methods. Using it is very simple:

In [33]:
import torch.optim as optim

# Create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop
optimizer.zero_grad()     # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()     # Does the update

size =  torch.Size([16, 6, 6])
